<a href="https://colab.research.google.com/github/QiaoLin22/MASTER-LLM-DL/blob/main/huggingface_model_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.17.0  # Ensure the correct version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.


In [2]:
import torch
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    DistilBertTokenizer, DistilBertForSequenceClassification,
    BertTokenizer, BertForQuestionAnswering,
    BartTokenizer, BartForConditionalGeneration,
    MarianMTModel, MarianTokenizer,
    AutoTokenizer, AutoModelForSeq2SeqLM  # Add these
)

In [3]:
# 1. Text Generator (T5) - Initialize conversation
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
input_text = "Generate a short story about AI."
input_ids = t5_tokenizer.encode(input_text, return_tensors='pt')
gen_output = t5_model.generate(input_ids, max_length=200)
generated_text = t5_tokenizer.decode(gen_output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

Generated Text:
Generate a short story about AI.


In [4]:
# 2. Sentiment Analyzer (DistilBERT) - Analyze sentiment of generated text
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
sentiment_input_ids = distilbert_tokenizer.encode(generated_text, return_tensors='pt')
sentiment_output = distilbert_model(sentiment_input_ids)
sentiment = torch.argmax(sentiment_output.logits)
print("\nSentiment Analysis:")
print("Sentiment:", "Positive" if sentiment else "Negative")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map


Sentiment Analysis:
Sentiment: Negative


In [5]:
# 3. Question Answering (BERT) - Ask & answer based on sentiment
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
if sentiment:
    question = "What is the main character's goal in this positive story?"
else:
    question = "What causes the conflict in this negative story?"

# Encode inputs with attention mask
encoding = bert_tokenizer.encode_plus(
    generated_text + " " + question,
    max_length=512,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

qa_input_ids = encoding['input_ids']
qa_attention_mask = encoding['attention_mask']

qa_output = bert_model(qa_input_ids, attention_mask=qa_attention_mask)

# Extract answer
answer_start = torch.argmax(qa_output.start_logits)
answer_end = torch.argmax(qa_output.end_logits) + 1
answer_ids = qa_input_ids[:, answer_start:answer_end]
qa_answer = bert_tokenizer.convert_tokens_to_string(bert_tokenizer.convert_ids_to_tokens(answer_ids[0]))

print("\nQuestion Answering:")
print("Question:", question)
print("Answer:", qa_answer)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a


Question Answering:
Question: What causes the conflict in this negative story?
Answer: a short story about ai . what causes the conflict in this negative story ? [SEP]


In [6]:
# 4. Text Summarizer (Bart) - Summarize QA response
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
summarize_input_ids = bart_tokenizer.encode(qa_answer, return_tensors='pt')
summarize_output = bart_model.generate(summarize_input_ids, max_length=50)
summary = bart_tokenizer.decode(summarize_output[0], skip_special_tokens=True)
print("\nSummary:")
print(summary)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]


Summary:
a short story about ai. what causes the conflict in this negative story? [SEP] A short story on a negative story. What causes the Conflict in This Negative Story?. What Causes the Conflict In This negative story


In [7]:
# 5. Language Translator (MarianMT) - Translate summary to French
marian_model_name = "Helsinki-NLP/opus-mt-en-fr"
marian_model = MarianMTModel.from_pretrained(marian_model_name)
marian_tokenizer = MarianTokenizer.from_pretrained(marian_model_name)
translate_input_ids = marian_tokenizer.encode(summary, return_tensors='pt')
translate_output = marian_model.generate(translate_input_ids)
translated_summary = marian_tokenizer.decode(translate_output[0], skip_special_tokens=True)
print("\nTranslated Summary (French):")
print(translated_summary)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]


Translated Summary (French):
une courte histoire sur ai. quelles sont les causes du conflit dans cette histoire négative? [SEP] Une courte histoire sur une histoire négative. Quelles sont les causes du conflit dans cette histoire négative?. Quelles sont les causes du conflit dans cette histoire négative


In [8]:
# 6. Chatbot (DialoGPT) - Respond to translated summary
from transformers import AutoTokenizer, AutoModelForCausalLM  # Update the import

dialogpt_tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
dialogpt_model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')
chat_input_ids = dialogpt_tokenizer.encode(translated_summary, return_tensors='pt')
chat_output = dialogpt_model.generate(chat_input_ids, max_length=100)
chat_response = dialogpt_tokenizer.decode(chat_output[0], skip_special_tokens=True)
print("\nChatbot Response:")
print(chat_response)

Downloading:   0%|          | 0.00/614 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Chatbot Response:
une courte histoire sur ai. quelles sont les causes du conflit dans cette histoire négative? [SEP] Une courte histoire sur une histoire négative. Quelles sont les causes du conflit dans cette histoire négative?. Quelles sont les causes du conflit dans cette histoire négative.


In [9]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.


In [10]:
from transformers import pipeline

# 1. Text Generation (GPT-2)
generator = pipeline('text-generation', model='gpt2', max_length=50)
initial_prompt = "The future of artificial intelligence is"
generated_text = generator(initial_prompt)[0]['generated_text']

print(f"Generated Text: {generated_text}")

# 2. Summarization (t5-small)
summarizer = pipeline('summarization', model='t5-small', max_length=50)
summarized_text = summarizer(generated_text)[0]['summary_text']

print(f"Summarized Text: {summarized_text}")

# 3. Question Answering (DistilBERT)
qa = pipeline('question-answering', model='distilbert-base-uncased-distilled-squad')
question = "What is the text about?"
qa_output = qa(question=question, context=summarized_text)['answer']

print(f"Question Answering Output: {qa_output}")

# 4. Translation (Helsinki-NLP, English to French)
translator = pipeline('translation_en_to_fr', model='Helsinki-NLP/opus-mt-en-fr')
translated_output = translator(qa_output)[0]['translation_text']

print(f"Translated Output (French): {translated_output}")

# 5. Sentiment Analysis (DistilBERT)
sentiment_analyzer = pipeline('sentiment-analysis')
sentiment = sentiment_analyzer(translated_output)[0]['label']

print(f"Sentiment of the Translated Text: {sentiment}")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Sentiment of the Translated Text: POSITIVE


In [11]:
# 1. Text Generation (GPT-2)
generator = pipeline('text-generation', model='gpt2', max_length=100)
initial_prompt = "The future of artificial intelligence is"
generated_text = generator(initial_prompt)[0]['generated_text']

print(f"Generated Text: {generated_text}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: The future of artificial intelligence is likely a lot more complicated than just how you get software to do something. It's likely we'll likely have plenty of reasons for doing something like that—just as we did for machine intelligence and the rest of the physics world. In fact, one thing we've come to expect with artificial intelligence lies mostly in this point, which is in a much bigger sense. AI's past-tense learning algorithms have always been capable of moving people by only a few feet,


In [12]:
# 2. Summarization (t5-small)
summarizer = pipeline('summarization', model='t5-small', max_length=50)
summarized_text = summarizer(generated_text)[0]['summary_text']

print(f"Summarized Text: {summarized_text}")

Summarized Text: the future of artificial intelligence is likely a lot more complicated than just how you get software to do something . one thing we've come to expect with artificial intelligence lies mostly in this point, which is in a much bigger sense 


In [13]:
# 3. Question Answering (DistilBERT)
qa = pipeline('question-answering', model='distilbert-base-uncased-distilled-squad')
question = "What is the text about?"
qa_output = qa(question=question, context=summarized_text)['answer']

print(f"Question Answering Output: {qa_output}")

Question Answering Output: artificial intelligence


In [14]:
# 4. Translation (Helsinki-NLP, English to French)
translator = pipeline('translation_en_to_fr', model='Helsinki-NLP/opus-mt-en-fr')
translated_output = translator(qa_output)[0]['translation_text']

print(f"Translated Output (French): {translated_output}")

Translated Output (French): intelligence artificielle


In [15]:
# 5. Sentiment Analysis (DistilBERT)
sentiment_analyzer = pipeline('sentiment-analysis')
sentiment = sentiment_analyzer(translated_output)[0]['label']

print(f"Sentiment of the Translated Text: {sentiment}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Sentiment of the Translated Text: POSITIVE
